In [ ]:
!pip install --no-index --find-links /kaggle/input/yolo11-seg-config-ultralytics/ultralytics ultralytics "numpy<2"

In [ ]:
import os
import cv2
import json
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO

In [ ]:
TEST_IMAGES_DIR = '/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images'
SAMPLE_SUBMISSION_PATH = '/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv'
MODEL_PATH = '/kaggle/input/yolo11-seg-config-ultralytics/runs/segment/train/weights/best.pt'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(MODEL_PATH).to(device)
model.eval()

def rle_encode(mask: np.ndarray, fg_val: int = 1) -> str:
    pixels = mask.flatten()
    dots = np.where(pixels == fg_val)[0]
    
    if len(dots) == 0:
        return "authentic"
    
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend([b + 1, 0])
        run_lengths[-1] += 1
        prev = b
    
    return json.dumps([int(x) for x in run_lengths])

In [ ]:
# Inference on test images
predictions = {}
test_files = sorted(os.listdir(TEST_IMAGES_DIR))

for file in tqdm(test_files, desc="Inference"):
    case_id = os.path.splitext(file)[0]
    img_path = os.path.join(TEST_IMAGES_DIR, file)
    
    # Uploading an image
    image = Image.open(img_path).convert('RGB')
    original_size = np.array(image).shape[:2]
    
    with torch.no_grad():
        results = model(image, imgsz=512, verbose=False)

    result = results[0]
    masks = result.masks
    boxes = result.boxes

    if masks is None or len(masks) == 0:
        predictions[case_id] = "authentic"
        continue

    # Confidence filtering
    confs = boxes.conf.cpu().numpy()
    mask_array = masks.data.cpu().numpy()

    # Confidence threshold
    CONF_THRESH = 0.6
    valid = confs > CONF_THRESH

    if not np.any(valid):
        predictions[case_id] = "authentic"
        continue

    # Combining all valid masks
    combined_mask_resized = np.zeros((original_size[0], original_size[1]), dtype=np.uint8)

    for i in np.where(valid)[0]:
        mask = mask_array[i]
        # Resize the mask to the original size
        resized_mask = cv2.resize(mask, (original_size[1], original_size[0]), interpolation=cv2.INTER_NEAREST)
        combined_mask_resized = np.logical_or(combined_mask_resized, resized_mask > 0.5)

    combined_mask_resized = combined_mask_resized.astype(np.uint8)

    if combined_mask_resized.sum() == 0:
        predictions[case_id] = "authentic"
    else:
        rle = rle_encode(combined_mask_resized)
        predictions[case_id] = rle

In [ ]:
# Create submission
sample_sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission_rows = []

for _, row in sample_sub.iterrows():
    case_id = str(row['case_id'])
    annotation = predictions.get(case_id, "authentic")
    submission_rows.append({'case_id': row['case_id'], 'annotation': annotation})

submission = pd.DataFrame(submission_rows)
submission.to_csv('submission.csv', index=False)

In [ ]:
FORGED_DIR = '/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged'
MASKS_DIR = '/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks'
MODEL_PATH = '/kaggle/input/yolo11-seg-config-ultralytics/runs/segment/train/weights/best.pt'

# Getting a list of forged images with masks
forged_files = [f for f in os.listdir(FORGED_DIR) if f.endswith(('.png', '.jpg', '.jpeg'))]

# We will leave only those that have an .npy mask
valid_files = []
for f in forged_files:
    base = os.path.splitext(f)[0]
    if os.path.exists(os.path.join(MASKS_DIR, f"{base}.npy")):
        valid_files.append(f)

In [ ]:
random.seed(42)
selected = random.sample(valid_files, min(5, len(valid_files))) # Take 5 random

fig, axes = plt.subplots(len(selected), 4, figsize=(16, 4 * len(selected)))
if len(selected) == 1:
    axes = [axes]

for idx, file in enumerate(selected):
    base_name = os.path.splitext(file)[0]
    img_path = os.path.join(FORGED_DIR, file)
    mask_path = os.path.join(MASKS_DIR, f"{base_name}.npy")

    # Uploading an image
    image = Image.open(img_path).convert('RGB')
    image_np = np.array(image)
    h_orig, w_orig = image_np.shape[:2]
    
    # Loading the GT mask
    gt_mask = np.load(mask_path)
    if gt_mask.ndim == 3:
        if gt_mask.shape[0] <= 10:
            gt_mask = np.any(gt_mask, axis=0)
        elif gt_mask.shape[-1] <= 10:
            gt_mask = np.any(gt_mask, axis=-1)
        else:
            gt_mask = gt_mask[0] if gt_mask.shape[0] <= 10 else gt_mask[:, :, 0]
    gt_mask = (gt_mask > 0).astype(np.uint8)
    
    # Model prediction
    with torch.no_grad():
        results = model(image, imgsz=512, verbose=False)
    pred_mask = np.zeros((h_orig, w_orig), dtype=np.uint8)
    
    if results[0].masks is not None:
        masks = results[0].masks.data.cpu().numpy()
        confs = results[0].boxes.conf.cpu().numpy()
        CONF_THRESH = 0.6
        for i, conf in enumerate(confs):
            if conf > CONF_THRESH:
                resized = cv2.resize(masks[i], (w_orig, h_orig), interpolation=cv2.INTER_NEAREST)
                pred_mask = np.logical_or(pred_mask, resized > 0.5)

    pred_mask = pred_mask.astype(np.uint8)

    # Overlay GT
    overlay_gt = image_np.copy()
    overlay_gt[gt_mask == 1] = [0, 255, 0]  # green

    # Overlay Pred
    overlay_pred = image_np.copy()
    overlay_pred[pred_mask == 1] = [255, 0, 0]  # red

    # Отображение
    axes[idx][0].imshow(image_np)
    axes[idx][0].set_title("Original")
    axes[idx][0].axis("off")

    axes[idx][1].imshow(gt_mask, cmap='gray')
    axes[idx][1].set_title("Ground Truth Mask")
    axes[idx][1].axis("off")

    axes[idx][2].imshow(pred_mask, cmap='gray')
    axes[idx][2].set_title("Predicted Mask")
    axes[idx][2].axis("off")

    axes[idx][3].imshow(overlay_gt)
    axes[idx][3].imshow(overlay_pred, alpha=0.5)  # let's put a predicate on top of GT
    axes[idx][3].set_title("Overlay: GT (green) + Pred (red)")
    axes[idx][3].axis("off")

plt.tight_layout()
plt.show()